In [2]:
import glob
import skimage.io as io
import numpy as np
#import commonfunctions as cf
import sklearn
from skimage.feature import hog
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
import pickle
import cv2
import random
import os
from PIL import Image, ImageEnhance
from PIL import Image, ImageEnhance
import cv2
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_val_predict, RandomizedSearchCV
from sklearn.metrics import *
from sklearn.decomposition import PCA

import pandas as pd



/home/mohamed/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [ ]:

'''
if you want to load the data again to work with them
'''

male_data=np.load('male_data_small.npy',allow_pickle=True)
female_data = np.load('female_data_small.npy',allow_pickle=True)

In [3]:
from skimage import feature, exposure

def HistogramofOrientedGradients(image):
    resized_img = resize(image, (128, 64))
    fd, hog_image = feature.hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(2, 2), visualize=True)
    
    return (np.array(fd)).flatten()



In [4]:
# print(np.reshape(HistogramofOrientedGradients(male_data[0]),(1,-1)).shape)

# temp=HistogramofOrientedGradients(male_data[0])
# X=temp
# X=np.reshape(X,(-1,1))
# pca=PCA(n_components=10)
# X=pca.fit_transform(X)
# print(X)

In [5]:


N_ANGLE_BINS = 40
BIN_SIZE = 360 // N_ANGLE_BINS
LEG_LENGTH = 25

def get_contour_pixels(bw_image):
    contours, _= cv2.findContours(
        bw_image, cv2.RETR_TREE, 
        cv2.CHAIN_APPROX_NONE
        ) 
    # contours = imutils.grab_contours(contours)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[1:]
    
    img2 = bw_image.copy()[:,:,np.newaxis]
    img2 = np.concatenate([img2, img2, img2], axis = 2)
    
    # if self.show_images:
    #     for cnt in contours : 
    #         cv2.drawContours(img2, [cnt], 0, (255, 0, 0), 1)  
            
    #     plt.imshow(img2, cmap='gray')
    return contours

def get_hinge_features(bw_image):
    # if self.is_binary:
    #     bw_image, _ = self.preprocess_binary_image(img_file, self.sharpness_factor, self.bordersize)
    # else:
    #     bw_image, _ = self.preprocess_image(img_file, self.sharpness_factor, self.bordersize)
    
    contours = get_contour_pixels(bw_image)
    
    hist = np.zeros((N_ANGLE_BINS, N_ANGLE_BINS))
        
    # print([len(cnt) for cnt in contours])
    for cnt in contours:
        n_pixels = len(cnt)
        if n_pixels <= LEG_LENGTH:
            continue
        
        points = np.array([point[0] for point in cnt])
        xs, ys = points[:, 0], points[:, 1]
        point_1s = np.array([cnt[(i + LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        point_2s = np.array([cnt[(i - LEG_LENGTH) % n_pixels][0] for i in range(n_pixels)])
        x1s, y1s = point_1s[:, 0], point_1s[:, 1]
        x2s, y2s = point_2s[:, 0], point_2s[:, 1]
        
        phi_1s = np.degrees(np.arctan2(y1s - ys, x1s - xs) + np.pi)
        phi_2s = np.degrees(np.arctan2(y2s - ys, x2s - xs) + np.pi)
        
        indices = np.where(phi_2s > phi_1s)[0]
        
        for i in indices:
            phi1 = int(phi_1s[i] // BIN_SIZE) % N_ANGLE_BINS
            phi2 = int(phi_2s[i] // BIN_SIZE) % N_ANGLE_BINS
            hist[phi1, phi2] += 1
            
    normalised_hist = hist / np.sum(hist)
    feature_vector = normalised_hist[np.triu_indices_from(normalised_hist, k = 1)]
    
    return feature_vector


#Initialization:
N_RHO_BINS = 7
N_ANGLE_BINS = 12
N_BINS = N_RHO_BINS * N_ANGLE_BINS
BIN_SIZE = 360 // N_ANGLE_BINS
R_INNER = 5.0
R_OUTER = 35.0
K_S = np.arange(3, 8)
def get_cold_features(bw_image, approx_poly_factor = 0.01 , sharpness_factor = 10 , bordersize = 3):

    #bw_image, _ = preprocess_image(img_file, sharpness_factor, bordersize)
    contours = get_contour_pixels(bw_image)
    
    rho_bins_edges = np.log10(np.linspace(R_INNER, R_OUTER, N_RHO_BINS))
    feature_vectors = np.zeros((len(K_S), N_BINS))
    
    for j, k in enumerate(K_S):
        hist = np.zeros((N_RHO_BINS, N_ANGLE_BINS))
        for cnt in contours:
            epsilon = approx_poly_factor * cv2.arcLength(cnt,True)
            cnt = cv2.approxPolyDP(cnt,epsilon,True)
            n_pixels = len(cnt)
            
            point_1s = np.array([point[0] for point in cnt])
            x1s, y1s = point_1s[:, 0], point_1s[:, 1]
            point_2s = np.array([cnt[(i + k) % n_pixels][0] for i in range(n_pixels)])
            x2s, y2s = point_2s[:, 0], point_2s[:, 1]
            
            thetas = np.degrees(np.arctan2(y2s - y1s, x2s - x1s) + np.pi)
            rhos = np.sqrt((y2s - y1s) ** 2 + (x2s - x1s) ** 2)
            if(rhos.all()):
                rhos_log_space = np.log10(rhos)
            else:
                rhos_log_space = rhos
            
            quantized_rhos = np.zeros(rhos.shape, dtype=int)
            for i in range(N_RHO_BINS):
                quantized_rhos += (rhos_log_space < rho_bins_edges[i])
                
            for i, r_bin in enumerate(quantized_rhos):
                theta_bin = int(thetas[i] // BIN_SIZE) % N_ANGLE_BINS
                hist[r_bin - 1, theta_bin] += 1
            
        normalised_hist = hist / hist.sum()
        feature_vectors[j] = normalised_hist.flatten()
        
    return feature_vectors.flatten()

    

In [6]:
print(get_hinge_features(male_data[0]).shape)
print(male_data[0].shape)

(66,)
(1420, 1000)


In [7]:
print(get_cold_features(male_data[0]).shape)
print(get_hinge_features(male_data[0]).shape)

(420,)
(66,)


In [8]:
a=np.array([1,2,3])
b=np.array([4,5,6])
c=np.array([4,5,6])
d=np.concatenate((a,b,c),axis=0)
print(d)

[1 2 3 4 5 6 4 5 6]


In [38]:


cold_male_feature=[]
cold_female_feature=[]
hinge_male_feature=[]
hinge_female_feature=[]
hog_male_feature=[]
hog_female_feature=[]
# pca=PCA(n_components=70)
# X=pca.fit_transform(X)
# print(X.shape, Y.shape)

#print(get_hinge_features(male_data[0]).shape)
for i,img in enumerate(male_data):
    
    cold_male_feature.append(get_cold_features(img))
    hinge_male_feature.append(get_hinge_features(img))
    hog_male_feature.append(HistogramofOrientedGradients(img))
    #male_feature.append(np.concatenate((cold,hinge),axis=0))
    # print(i)

for i,img in enumerate(female_data):
    
    cold_female_feature.append(get_cold_features(img))
    hinge_female_feature.append(get_hinge_features(img))
    hog_female_feature.append(HistogramofOrientedGradients(img))
    #female_feature.append(np.concatenate((cold,hinge),axis=0))

cold_male_feature=np.array(cold_male_feature)
hinge_male_feature=np.array(hinge_male_feature)
cold_female_feature=np.array(cold_female_feature)
hinge_female_feature=np.array(hinge_female_feature)
hog_male_feature=np.array(hog_male_feature)
hog_female_feature=np.array(hog_female_feature)

In [39]:

X_cold=np.concatenate((cold_male_feature,cold_female_feature),axis=0)
pca=PCA(n_components=70)
X_cold=pca.fit_transform(X_cold)

X_hinge=np.concatenate((hinge_male_feature,hinge_female_feature),axis=0)

X_hog=np.concatenate((hog_male_feature,hog_female_feature),axis=0)

X=np.concatenate((X_cold,X_hinge,X_hog),axis=1)


Y=np.concatenate((np.ones(cold_male_feature.shape[0]),
                np.zeros(cold_female_feature.shape[0])),
                axis=0)

print(X_cold[:,:70].shape)
print(X_hinge.shape)
print(X.shape)   


(363, 70)
(363, 66)
(363, 3916)


In [10]:
print(male_data.shape)    
print(female_data.shape)    

(232, 1420, 1000)
(131, 1420, 1000)


In [11]:
pca=PCA(n_components=70)
X=pca.fit_transform(X)
print(X.shape, Y.shape)


(363, 70) (363,)


In [40]:
X,Y=sklearn.utils.shuffle(X,Y)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=42) # 70% training and 30% test


In [41]:
model_cold = svm.SVC(kernel='rbf', verbose=True, C=10,probability=True)

model_cold.fit(X_train[:,:70], y_train)
y_pred_cold=model_cold.predict(X_test[:,:70])
y_pred_prob_cold=model_cold.predict_proba(X_test[:,:70])
print(y_pred_prob_cold)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_cold))


[LibSVM]..*
optimization finished, #iter = 689
obj = -220.782953, rho = -0.504807
nSV = 163, nBSV = 1
Total nSV = 163
.*.*
optimization finished, #iter = 687
obj = -232.373838, rho = -0.383282
nSV = 157, nBSV = 2
Total nSV = 157
.*.*
optimization finished, #iter = 665
obj = -221.806968, rho = -0.808826
nSV = 157, nBSV = 2
Total nSV = 157
..*
optimization finished, #iter = 691
obj = -238.461565, rho = -0.395106
nSV = 156, nBSV = 3
Total nSV = 156
.*.*
optimization finished, #iter = 647
obj = -227.865327, rho = -0.562353
nSV = 156, nBSV = 1
Total nSV = 156
..*
optimization finished, #iter = 849
obj = -275.310715, rho = 0.578309
nSV = 188, nBSV = 2
Total nSV = 188
[[9.13145206e-07 9.99999087e-01]
 [2.29057635e-01 7.70942365e-01]
 [9.95239933e-01 4.76006737e-03]
 [1.04636253e-01 8.95363747e-01]
 [4.57510717e-01 5.42489283e-01]
 [9.49738382e-01 5.02616181e-02]
 [6.82961077e-01 3.17038923e-01]
 [8.89054360e-01 1.10945640e-01]
 [3.46176786e-03 9.96538232e-01]
 [7.47935163e-02 9.25206484e-01]


In [42]:
model_hinge = svm.SVC(kernel='rbf', verbose=True, C=10,probability=True)

model_hinge.fit(X_train[:,70:136], y_train)
y_pred_hinge=model_hinge.predict(X_test[:,70:136])
y_pred_prob_hinge=model_hinge.predict_proba(X_test[:,70:136])
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_hinge))


[LibSVM].*.*
optimization finished, #iter = 587
obj = -968.319472, rho = -1.926764
nSV = 160, nBSV = 98
Total nSV = 160
.*.*
optimization finished, #iter = 526
obj = -914.915182, rho = -1.540743
nSV = 149, nBSV = 92
Total nSV = 149
.*.*
optimization finished, #iter = 553
obj = -922.296063, rho = -1.926448
nSV = 152, nBSV = 95
Total nSV = 152
.*.*
optimization finished, #iter = 557
obj = -995.961121, rho = -1.884385
nSV = 161, nBSV = 100
Total nSV = 161
.*.*
optimization finished, #iter = 628
obj = -1012.424396, rho = -1.453762
nSV = 166, nBSV = 98
Total nSV = 166
.*.*
optimization finished, #iter = 717
obj = -1204.491403, rho = 1.785061
nSV = 194, nBSV = 126
Total nSV = 194
Accuracy: 0.8356164383561644


In [45]:
model_hog = svm.SVC(kernel='rbf', verbose=True, C=10,probability=True)

model_hog.fit(X_train[:,136:], y_train)
y_pred_hog=model_hog.predict(X_test[:,136:])
print(y_pred_hog)
y_pred_prob_hog=model_hog.predict_proba(X_test[:,136:])
print("Accuracy:",metrics.accuracy_score(y_test, y_pred_hog))

[LibSVM].*.*
optimization finished, #iter = 521
obj = -202.207614, rho = -0.545953
nSV = 226, nBSV = 0
Total nSV = 226
.*.*
optimization finished, #iter = 515
obj = -190.939916, rho = -0.667978
nSV = 225, nBSV = 0
Total nSV = 225
.*.*
optimization finished, #iter = 509
obj = -199.549010, rho = -0.690826
nSV = 228, nBSV = 0
Total nSV = 228
.*.*
optimization finished, #iter = 522
obj = -192.886691, rho = -0.580548
nSV = 220, nBSV = 0
Total nSV = 220
.*.*
optimization finished, #iter = 498
obj = -191.762863, rho = -0.602087
nSV = 221, nBSV = 0
Total nSV = 221
.*.*
optimization finished, #iter = 651
obj = -247.986771, rho = 0.631909
nSV = 275, nBSV = 0
Total nSV = 275
[1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0.
 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0.
 1.]
Accuracy: 0.6986301369863014


In [49]:
y_pred=np.argmax(y_pred_prob_hinge+y_pred_prob_cold+y_pred_prob_hog, axis=1)
print(y_pred)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

[1 1 0 1 0 0 1 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 0 1
 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0]
Accuracy: 0.8493150684931506


In [37]:

y_pred = clf.predict(X_test)

# print(y_pred)
# print(y_test)

NameError: name 'clf' is not defined

In [15]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8356164383561644


In [20]:
accuracy=0
X=np.concatenate((male_feature,female_feature),axis=0)
Y=np.concatenate((np.ones(male_feature.shape[0]),
                    np.zeros(female_feature.shape[0])),
                    axis=0)
pca=PCA(n_components=200)
X=pca.fit_transform(X)
t=0
for i in range(0,100):


    X,Y=sklearn.utils.shuffle(X,Y)

    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=(i+1000)%5) # 70% training and 30% test
    clf = svm.SVC(kernel='rbf', verbose=True, C=10)

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    accuracy+=metrics.accuracy_score(y_test, y_pred)

    
    pass
print(accuracy/100)

[LibSVM].*.*
optimization finished, #iter = 761
obj = -224.191931, rho = 0.573204
nSV = 223, nBSV = 0
Total nSV = 223
[LibSVM].*.*
optimization finished, #iter = 702
obj = -219.552534, rho = 0.602717
nSV = 215, nBSV = 1
Total nSV = 215
[LibSVM].*.*
optimization finished, #iter = 721
obj = -217.396874, rho = 0.741653
nSV = 213, nBSV = 1
Total nSV = 213
[LibSVM].*.*
optimization finished, #iter = 740
obj = -255.895017, rho = 0.698693
nSV = 219, nBSV = 1
Total nSV = 219
[LibSVM].*.*
optimization finished, #iter = 797
obj = -232.070026, rho = 0.565015
nSV = 223, nBSV = 1
Total nSV = 223
[LibSVM].*.*
optimization finished, #iter = 738
obj = -223.555646, rho = 0.617828
nSV = 207, nBSV = 0
Total nSV = 207
[LibSVM].*.*
optimization finished, #iter = 809
obj = -240.455477, rho = 0.564678
nSV = 232, nBSV = 1
Total nSV = 232
[LibSVM].*.*
optimization finished, #iter = 782
obj = -227.563437, rho = 0.541761
nSV = 227, nBSV = 0
Total nSV = 227
[LibSVM].*.*
optimization finished, #iter = 789
obj = -2

In [34]:
print(male_feature.shape)

(232, 486)


In [ ]:
a=np.ones((3,3))
b=np.zeros((3,4))
print(a.shape,b.shape)
print(np.concatenate((a,b),axis=1))

(3, 3) (3, 4)
[[1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0.]]
